In [ ]:
from IPython.display import display
import IPython
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import sklearn as skl
%matplotlib notebook
%matplotlib inline
import tensorflow as tf
!pip install mglearn
!pip install gensim
!pip install nltk
import nltk

import mglearn as mglearn
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

     |████████████████████████████████| 542kB 4.3MB/s 
  Created wheel for mglearn: filename=mglearn-0.1.9-py2.py3-none-any.whl size=582638 sha256=a28b30c8377857e0a167d1dfb23390b92805f8bb557f7494fb22820860343dc0
  Stored in directory: /root/.cache/pip/wheels/eb/a6/ea/a6a3716233fa62fc561259b5cb1e28f79e9ff3592c0adac5f0
Successfully built mglearn


In [ ]:
nltk.download('stopwords')
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving sample_submission.csv to sample_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv
User uploaded file "sample_submission.csv" with length 537111 bytes
User uploaded file "test.csv" with length 1351241 bytes
User uploaded file "train.csv" with length 3295644 bytes


In [ ]:
sample = pd.read_csv('/content/sample_submission.csv')
test = pd.read_csv('/content/test.csv')
train = pd.read_csv('/content/train.csv')

In [ ]:
print('[Train.shape:{}]'.format(train.shape))
print(type(train))
print('\n[Test.shape:{}]'.format(test.shape))
print(type(test))

[Train.shape:(19579, 3)]
<class 'pandas.core.frame.DataFrame'>

[Test.shape:(8392, 2)]
<class 'pandas.core.frame.DataFrame'>


In [ ]:
def multiclass_logloss(actual, predicted, eps=1e-15):#eps 는 p=1, p=0일 때를 대비해서
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1: # (N,) 의 형태라면= 왜냐면 instance 별로 class를
          #나타내고 있으니깐!!! (N,)가 되는거야
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        #actual.shape[0]=N
        #predicted.shape=(N,c) : predicted.shape[1]=c
        for i, val in enumerate(actual): #actual=[0,1,0,1,,,] 이런식이잖아
            actual2[i, val] = 1 #val자리가 해당 클래스의 열이기에 거기다가 1을 배치시킴
        actual = actual2 #(N,c) one hot encoding matrix 만든거임

    clip = np.clip(predicted, eps, 1 - eps) #예측한 확률이 0이나 1을 못 만들게 하기 위해
    rows = actual.shape[0] #N instance 개수
    vsota = np.sum(actual * np.log(clip)) 
    #actual*np.log(clip)=N*c 행렬이 나옴
    #거기다 sum은 N*c짜리 원소를 모두 더함
    
    return -1.0 / rows * vsota
    #결론 적으로 loss값이 딱 하나 나옴


In [ ]:
p=[[0.5,0.3,0.2],[0.1,0.2,0.7],[0.3,0.4,0.3]]
a=[[1,0,0],[0,0,1],[1,0,0]]
len(p)

3

In [ ]:
log_p=np.log(p)
log_p
print(a*log_p)

loss= -1 /len(a)*(np.sum(a*log_p))
loss=round(loss,4)
loss

[[-0.69314718 -0.         -0.        ]
 [-0.         -0.         -0.35667494]
 [-1.2039728  -0.         -0.        ]]


0.7513

In [ ]:
a=np.array([[1],[2],[3],[4],[5]])
b=np.array([1,2,3,4,5])
print(len(a.shape))
print(len(b.shape))
print(a.shape)
print(b.shape)
print(b.shape[0])

2
1
(5, 1)
(5,)
5


In [ ]:
lbl_enc = skl.preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)
print(y)
print(y.shape)

[0 1 0 ... 0 0 1]
(19579,)


In [ ]:
xtrain, xvalid, ytrain, yvalid = skl.model_selection.train_test_split(train.text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [ ]:
#다 넘파이 배열로 만들어 버렸네
print(type(train.text)) #type이 series
print(train.text)
print (train.text.shape)

print(type(train.text.values)) #type이 넘파이 배열
print(train.text.values)
print (train.text.values.shape)

print (xtrain.shape)
print(xtrain)
print (xvalid.shape)
print(xvalid)
print (ytrain.shape)
print(ytrain)
print (yvalid.shape)
print(yvalid)

<class 'pandas.core.series.Series'>
0        This process, however, afforded me no means of...
1        It never once occurred to me that the fumbling...
2        In his left hand was a gold snuff box, from wh...
3        How lovely is spring As we looked from Windsor...
4        Finding nothing else, not even gold, the Super...
                               ...                        
19574    I could have fancied, while I looked at it, th...
19575    The lids clenched themselves together as if in...
19576    Mais il faut agir that is to say, a Frenchman ...
19577    For an item of news like this, it strikes us i...
19578    He laid a gnarled claw on my shoulder, and it ...
Name: text, Length: 19579, dtype: object
(19579,)
<class 'numpy.ndarray'>
['This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall.'
 'It n

# 텍스트의 수치화 : CTV=DTM, TFV=TFIDF,SVD=TFIDF에 SVD적용

In [ ]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

In [ ]:
#CTV=DTM, .TFV=TFIDF,.SVD=TFIDF에 SVD적용
# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain)  #이미 벡터와 시킨거지
xvalid_ctv = ctv.transform(xvalid)

# Since SVMs take a lot of time, we will reduce the number of features from the TF-IDF 
# using Singular Value Decomposition before applying SVM.
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# Logistic Regression

In [ ]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0, max_iter=500)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)
print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.572 


In [ ]:
# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)
print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.527 


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


# Naiive Bayesian

In [ ]:
# Fitting a simple Naive Bayes on TFIDF
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.578 


In [ ]:
# Fitting a simple Naive Bayes on Counts
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.485 


# SVM

In [ ]:
# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [ ]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))


logloss: 0.732 


# XGBoost

In [ ]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

In [ ]:
# Fitting a simple xgboost on Counts
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

In [ ]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))